# Analiza 'nocnej promocji' w sklepie euro.com.pl

## 1. Wprowadzenie

### 1.1 Cel analizy

<p>
Analiza promocji 'nocna promocja' dostępnej w sklepie internetowym euro.com.pl, ma na celu poznanie czy jseteśmy w stanie odkryć jakie produkty / kategorie produktów są częściej przeceniane.
</p>
<p>
Postaram się znaleźć odpowiedź na pytanie czy warto codziennie przeglądać tą promocję, czy produkty się zmieniają, czy sklep 'odgrzewa stare kotlety'.
</p>

### 1.2 O zbiorze danych

<p>
Dane analizowane w tym miejscu pochodzą ze sklepu internetowego euro.com.pl, a zbierane były za pomocą autorskiego skryptu scrapującego napisanego w Pythonie. Dane ze strony pobierane były przez 7 dni - od 25.01.2022 r. do 31.01.2022 r. Skrypt scrapujący oraz zebrane pliki w formacie csv z danymi podzielonymi na poszczególne dni znajdują się w folderze 'data' tego repozytorium.
</p>
<p>
    Zebrane dane o promocji wzbogacone zostaną o dane doytczące kategorii produktów dostępnych w tym sklepie, a których analiza znajduje się w pliku 'category_analysis' będącym w tym repozytorium.
</p>

## 2. Przygotowanie zbioru

### 2.1 Wczytanie potrzebnych bibliotek oraz plików z danymi

#### Niezbędne biblioteki:

In [40]:
import os
import pandas as pd
import matplotlib.pyplot as plt

#### Wczytanie plików z danymi oraz połączenie z danymi dot. kategorii znajdujących się w sklepie:

In [41]:
pliki_z_danymi = []
storage_path = './data'

for item in os.listdir(storage_path):
	if item.endswith(".csv"):
		pliki_z_danymi.append(f'{storage_path}/{item}')

df = pd.DataFrame()

for plik in pliki_z_danymi:
	df = pd.concat([df, pd.read_csv(plik, header=0, parse_dates=['scraping_time'])], ignore_index=True)

df_categories = pd.read_csv('./categories/categories.csv', names=['parent_category', 'category', 'link_to_category', 'how_many_offers'])

df = df.merge(df_categories, on='category')

#### Nazwy zmiennych

<p>
Na etapie zbierania danych zadbano o prawidłowe nazwanie zmiennych, aby łatwo można było się zorientować, z czym mamy do czynienia, dlatego nie ma potrzeby zmian.
</p>

### 2.2 Zmiana typów zmiennych

In [42]:
df.dtypes

product_id                 float64
plu                        float64
regular                    float64
discount                   float64
discount_amount            float64
discount_proc              float64
discount_code               object
availability                object
href                        object
category                    object
domain_name                 object
scraping_time       datetime64[ns]
parent_category             object
link_to_category            object
how_many_offers             object
dtype: object

#### Przegląd typów zmiennych:
<ul>
    <li>
        'product_id', 'plu' - z wiedzy eksperckiej ;) wiadomo, że są to identyfikatory poszczególnych ofert dlatego postanowiłem zmienić ich typ na 'int', choć nie jest to operacja niezbędna,
    </li>
    <li>
        zmienne: 'regular' do 'discount_proc' poprawnie są interpretowane przez Python jako zmienne numeryczne - nie mogło być inaczej, ponieważ już na etapie zbirania danych zadbano o ich prawidłowe sładowanie,
    </li>
    <li>
        'availability', 'href', 'category', 'domain_name', 'parent_category', 'link_to_category' - są to zmienne kategoryczne, interpretowane przeez Python jako 'object',
    </li>
    <li>
        'scraping_time' - zmienna typu kategorycznego poprawnie zinterpretowana jako datę.
    </li>
    <li>
        'how_many_offers' - zmienna liczbowa, która powinna być typu 'int' - zostanie to poprawione w kolejnym kroku.
    </li>
</ul>

#### Zmiany dla potrzeb dalszej analizy:
<ol>
    <li>
        zmiana typu zmiennych 'plu' oraz 'how_many_offers' do typu 'int',
    </li>
    <li>
        zastąpienie zmiennej 'scraping_time' na 'string' zawierający datę w formacie yyyy-mm-dd i nadanie jej nazwy 'date',
    </li>
    <li>
        usunięcie niepotrzebnych kolumn - dotyczy to zmiennych, które nie wniosą nic pożytecznego do analizy, a mogą nie potrzebnie przeszkadzać, są to:
        <ul>
            <li>'product_id' (mamy już identyfikator 'plu'),</li>
            <li>'href' (zmienna przechowywująca link do oferty),</li>
            <li>'domain_name' (nazwa domeny - pozostałość po procesie zbierania danych),</li>
            <li>'scraping_time' (nie potrzebna, bo zastępuję ją nową zmienną 'date'),</li>
            <li>'link_to_category' (jak nazwa mówi - link do kategorii, nie potrzebny),</li>
         </ul>
    </li>
</ol>

In [43]:
df.plu = df.plu.astype(int)
df.how_many_offers = df.how_many_offers.astype(float).astype(int)

In [44]:
df['date'] = df.scraping_time.dt.strftime('%Y-%m-%d')

In [45]:
df.drop(labels=['product_id', 'href', 'domain_name', 'scraping_time', 'link_to_category'], axis=1, inplace=True)

#### Po zmianach:

In [46]:
df.dtypes

plu                  int32
regular            float64
discount           float64
discount_amount    float64
discount_proc      float64
discount_code       object
availability        object
category            object
parent_category     object
how_many_offers      int32
date                object
dtype: object